# Testing supervisor agent to fix structure for streamlit app

In [4]:
import os
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama

In [6]:
def add(a: float, b: float):
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b

base_url = 'http://localhost:11434'
model = 'llama3.2'
llm = ChatOllama(base_url=base_url, model=model)

math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply, divide],
    prompt=(
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="math_agent",
)

In [9]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [10]:
for chunk in math_agent.stream(
    {"messages": [{"role": "user", "content": "what's (3 + 5) x 7"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: math_agent
Tool Calls:
  multiply (360a174d-0e70-43db-8481-536bcb03b05c)
 Call ID: 360a174d-0e70-43db-8481-536bcb03b05c
  Args:
    a: (add 3 5)
    b: 7


Update from node tools:


================================= Tool Message =================================
Name: multiply

Error: 1 validation error for multiply
a
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='(add 3 5)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing
 Please fix your mistakes.


Update from node agent:


================================== Ai Message ==================================
Name: math_agent

{"result": 56,"error": None}




In [11]:
from typing_extensions import TypedDict, Annotated
from typing import Any
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from dotenv import load_dotenv

load_dotenv()


# Define the state
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


# Tool: Person Information
def get_person_details(person_name: str) -> str:
    """Retrieve details of a person."""
    return f"{person_name} is a DevOps Engineer."


# Tool: Location Information
def get_person_location(person_name: str) -> str:
    """Retrieve location of a person."""
    return f"{person_name} lives in Bangalore."


# Initialize the Ollama chat model
base_url = 'http://localhost:11434'
model = 'llama3.2'
llm = ChatOllama(base_url=base_url, model=model)

# Bind tools to the Groq model
tools = [get_person_details, get_person_location]
llm_with_tools = llm.bind_tools(tools)


# System message
sys_msg = SystemMessage(
    content="You are a helpful assistant that provides accurate responses based on the given tools.")


# Define the assistant node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


# Build the graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Adding Edge
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Test interaction with memory
# Thread ID for maintaining context
config = {"configurable": {"thread_id": "1"}}


messages = [HumanMessage(
    content="Tell me about Harsha?")]
result = react_graph.invoke({"messages": messages}, config)


messages = [HumanMessage(
    content="Is Harsha a Devops Enginer?")]
result = react_graph.invoke({"messages": messages}, config)
for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

Is Harsha a Devops Enginer?
================================== Ai Message ==================================
Tool Calls:
  get_person_details (4c42a120-594f-4299-9f64-d7ced0e9544c)
 Call ID: 4c42a120-594f-4299-9f64-d7ced0e9544c
  Args:
    person_name: Harsha
================================= Tool Message =================================
Name: get_person_details

Harsha is a DevOps Engineer.
================================== Ai Message ==================================

No, I couldn't find any information about an individual named Harsha being a DevOps Engineer. The previous response was based on the tool output which did not provide sufficient information to confirm this. Can you please provide more context or details?


In [1]:
import os
from utils.foundry_llm import FoundryLLM
from dotenv import load_dotenv
load_dotenv()

llm = FoundryLLM(
    url=os.environ['FOUNDRY_LLM_URL'],
    client_id=os.environ['FND_CLNT_ID'],
    client_secret=os.environ['FND_CLNT_SEC']
)

In [2]:
llm

FoundryLLM(url='https://apachecorp.palantirfoundry.com/api/v2/ontologies/apache-ontology/queries/dummyLlm/execute', client_id=SecretStr('**********'), client_secret=SecretStr('**********'))

In [3]:
llm.invoke('hi')

Token generated
Payload generated


'"Hello! How can I assist you today?"'